# High-Level API: Simple Data Transfer

**Goal:** Transfer data with one function call. No complexity, no configuration hell.

This is what 90% of users need. Just works out of the box.

## Why This Matters

- **Simple**: One function call does everything
- **Safe**: Built-in limits and validation
- **Transparent**: Shows you what's happening
- **Smart**: Auto-derives destination names

## Real World Usage

Most business users just want to move data. They don't want to learn complex APIs or manage connections. This API is for them.


In [1]:
from snowpark_db_api import transfer, transfer_sample, transfer_with_validation

## 1. Simple Table Transfer

Copy entire tables with zero configuration.


In [2]:
# Simple query with just 100 rows for testing
result = transfer(
    "(SELECT TOP 100 * FROM dbo.RandomDataWith100Columns) AS high_level_test",
    snowflake_database="DB_API_MSSQL",
    snowflake_schema="DEV_TEST", 
    create_db_if_missing=True
)

🚀 Starting transfer: (SELECT TOP 100 * FROM dbo.RandomDataWith100Columns) AS high_level_test
🎯 Configuration overrides: database=DB_API_MSSQL, schema=DEV_TEST, create_db_if_missing=True
📋 Detected custom query
📊 Source: alingtestserver.database.windows.net (sqlserver)
❄️  Target: trb65519.DB_API_MSSQL
📝 Auto-derived destination: HIGH_LEVEL_TEST
2025-07-03 01:24:28 - snowpark_db_api.core - INFO - Setting up connections
2025-07-03 01:24:30 - snowpark_db_api.core - INFO - All connections established
2025-07-03 01:24:30 - snowpark_db_api.core - INFO - Starting transfer using custom query -> HIGH_LEVEL_TEST


DataFrameReader.dbapi() is in private preview since 1.29.0. Do not use it in production. 


2025-07-03 01:24:32 - snowpark_db_api.connections - INFO - Successfully connected to SQL Server: alingtestserver.database.windows.net
2025-07-03 01:24:32 - snowpark_db_api.connections - INFO - Successfully connected to SQL Server: alingtestserver.database.windows.net
2025-07-03 01:24:34 - snowpark_db_api.connections - INFO - Successfully connected to SQL Server: alingtestserver.database.windows.net
2025-07-03 01:24:38 - snowpark_db_api.core - INFO - Transferring 100 rows
2025-07-03 01:24:39 - snowpark_db_api.core - INFO - Transfer completed: 100 rows in 8.8s


What happened:
- Connected to SQL Server and Snowflake automatically
- Used DB_API_MSSQL database instead of environment default
- Used STAGING schema instead of PUBLIC
- Created PUBLIC table in the right place
- Closed connections cleanly

## 🎯 Runtime Configuration Examples

Perfect for analysts who need small adjustments without complexity!

```python

# Development environment
transfer("dbo.TestData", 
         snowflake_database="DB_API_DEV",
         snowflake_schema="SANDBOX",
         snowflake_warehouse="XSMALL_WH")

# Production environment  
transfer("dbo.ProductionData",
         snowflake_database="DB_API_PROD", 
         snowflake_schema="PUBLIC",
         snowflake_warehouse="LARGE_WH")

# Quick test in your personal schema
transfer("dbo.MyData",
         snowflake_database="DB_API_MSSQL",
         snowflake_schema="JOHN_SANDBOX")

```



## 🎯 Runtime Configuration Examples

Perfect for analysts who need small adjustments without complexity!


## 2. Query-Based Transfer

Transfer filtered data using real SQL queries. The destination is auto-derived from the query alias.


In [3]:
# Recent orders - destination auto-derived from alias
# Test with small sample - always safe
result = transfer_sample(
    "(SELECT TOP 500 ID, Column0, Column1, Column2, Column3 FROM dbo.RandomDataWith100Columns) AS sample_test",
    rows=50,  # Even smaller sample
    snowflake_database="DB_API_MSSQL",
    snowflake_schema="PUBLIC"
)

🧪 Sampling 50 rows for testing
🚀 Starting transfer: (SELECT TOP 500 ID, Column0, Column1, Column2, Column3 FROM dbo.RandomDataWith100Columns) AS sample_test
🎯 Configuration overrides: database=DB_API_MSSQL, create_db_if_missing=False (safe default), schema=PUBLIC
📋 Detected custom query
📊 Source: alingtestserver.database.windows.net (sqlserver)
❄️  Target: trb65519.DB_API_MSSQL
📝 Auto-derived destination: SAMPLE_TEST
🔢 Limited to 50 rows
2025-07-03 01:27:09 - snowpark_db_api.core - INFO - Setting up connections
2025-07-03 01:27:10 - snowpark_db_api.core - INFO - All connections established
2025-07-03 01:27:10 - snowpark_db_api.core - INFO - Starting transfer using custom query -> SAMPLE_TEST
2025-07-03 01:27:11 - snowpark_db_api.connections - INFO - Successfully connected to SQL Server: alingtestserver.database.windows.net
2025-07-03 01:27:12 - snowpark_db_api.connections - INFO - Successfully connected to SQL Server: alingtestserver.database.windows.net
2025-07-03 01:27:14 - snowpark_

## 3. Multi-Query Examples

In [4]:
queries = [
    # Small user profile data
    ("(SELECT TOP 10 Id, FullName, Country FROM dbo.UserProfile WHERE Id IS NOT NULL) AS user_sample", "User profile data"),
    
    # Order data with date filter
    ("(SELECT TOP 20 o_orderkey, o_custkey, o_totalprice FROM dbo.ORDERS WHERE o_orderdate >= '2020-01-01') AS recent_orders", "Recent orders"),
    
    # Random data with specific columns
    ("(SELECT TOP 25 ID, Column0, Column1, Column5 as timestamp_col FROM dbo.RandomDataWith100Columns) AS minimal_test", "Minimal column set"),
]

results = []
for query, description in queries:
    print(f"\n📝 Testing: {description}")
    result = transfer(
        query,
        snowflake_database="DB_API_MSSQL",
        snowflake_schema="PUBLIC",
        create_db_if_missing=False
    )
    results.append((description, result))


📝 Testing: User profile data
🚀 Starting transfer: (SELECT TOP 10 Id, FullName, Country FROM dbo.UserProfile WHERE Id IS NOT NULL) AS user_sample
🎯 Configuration overrides: database=DB_API_MSSQL, schema=PUBLIC, create_db_if_missing=False
📋 Detected custom query
📊 Source: alingtestserver.database.windows.net (sqlserver)
❄️  Target: trb65519.DB_API_MSSQL
📝 Auto-derived destination: USER_SAMPLE
2025-07-03 01:28:11 - snowpark_db_api.core - INFO - Setting up connections
2025-07-03 01:28:11 - snowpark_db_api.core - INFO - All connections established
2025-07-03 01:28:12 - snowpark_db_api.core - INFO - Starting transfer using custom query -> USER_SAMPLE
2025-07-03 01:28:12 - snowpark_db_api.connections - INFO - Successfully connected to SQL Server: alingtestserver.database.windows.net
2025-07-03 01:28:13 - snowpark_db_api.connections - INFO - Successfully connected to SQL Server: alingtestserver.database.windows.net
2025-07-03 01:28:14 - snowpark_db_api.connections - INFO - Successfully connec

## 4. Transfer with Validation

For production workloads, add validation to ensure data quality.


In [5]:
# Transfer with automatic validation
validation_rules = {
    'min_rows': 1,
    'max_rows': 100,
    'required_columns': ['ID', 'Column0']
}

result = transfer_with_validation(
    "(SELECT TOP 15 ID, Column0, Column1 FROM dbo.RandomDataWith100Columns) AS validated_test",
    validation_rules=validation_rules,
    snowflake_database="DB_API_MSSQL",
    snowflake_schema="PUBLIC",
    create_db_if_missing=False
)

print(f"Transfer validated: {result['transfer_success']}")
print(f"Pre-transfer checks: {result['pre_transfer']['valid']}")
print(f"Post-transfer checks: {result['post_transfer']}")


✅ Starting validated transfer
🚀 Starting transfer: (SELECT TOP 15 ID, Column0, Column1 FROM dbo.RandomDataWith100Columns) AS validated_test
🎯 Configuration overrides: database=DB_API_MSSQL, schema=PUBLIC, create_db_if_missing=False
📋 Detected custom query
📊 Source: alingtestserver.database.windows.net (sqlserver)
❄️  Target: trb65519.DB_API_MSSQL
📝 Auto-derived destination: VALIDATED_TEST
2025-07-03 01:28:59 - snowpark_db_api.core - INFO - Setting up connections
2025-07-03 01:29:00 - snowpark_db_api.core - INFO - All connections established
2025-07-03 01:29:00 - snowpark_db_api.core - INFO - Starting transfer using custom query -> VALIDATED_TEST
2025-07-03 01:29:01 - snowpark_db_api.connections - INFO - Successfully connected to SQL Server: alingtestserver.database.windows.net
2025-07-03 01:29:02 - snowpark_db_api.connections - INFO - Successfully connected to SQL Server: alingtestserver.database.windows.net
2025-07-03 01:29:03 - snowpark_db_api.connections - INFO - Successfully connec

## Summary: When to Use High-Level API

**Use this API when:**
- ✅ You want to transfer data quickly with minimal code
- ✅ You're okay with sensible defaults BUT need flexibility for small changes
- ✅ You want built-in safety features (limits, validation)
- ✅ You prefer transparency (see what's happening)
- ✅ **NEW: You need to override database/schema/warehouse at runtime**

**Don't use this API when:**
- ❌ You need custom connection pooling
- ❌ You want to control transform pipelines
- ❌ You need access to raw Snowpark DataFrames
- ❌ You're building complex data workflows

**🎯 What makes this API powerful:**
- **Simple**: One function call for most use cases
- **Flexible**: Override any configuration parameter at runtime
- **Safe**: Built-in limits and validation
- **Transparent**: Shows exactly what's happening
- **Environment-aware**: Works in dev, staging, prod with same code

**What you learned:**
- `transfer()` - Main function with runtime config overrides
- `transfer_sample()` - Safe testing with flexible targeting
- `transfer_with_validation()` - Production-ready with environment flexibility
- Query aliases automatically become destination names
- **NEW: Runtime configuration overrides for database, schema, warehouse**

**Next:** Check out the Mid-Level API for more control over your transfers.
